- 웹페이지의 종류
- 정적페이지 : 페이지의 데이터가 변경될 때 URL이 변경 : HTML
- 동적페이지 : 페이지의 데이터가 변경될 때 URL이 변경x : JSON (URL은 안바뀌고 화면만 바뀜)

- requests package
- 브라우저의 URL을 입력하면 서버에서 데이터를 다운받아 화면에 출력 : URL -> DATA
- requests 패키지 : URL -> DATA

In [1]:
import requests
import pandas as pd

* **코스피 정보 가져오기**

In [4]:
# 1. 웹서비스 분석 : 크롬 개발자 도구 : URL
# https://m.stock.naver.com
url = "https://m.stock.naver.com/api/index/KOSPI/price?pageSize=10&page=3"

In [5]:
# 2. request(URL) > response(json) : JSON(str)
response = requests.get(url)
response

<Response [200]>

In [ ]:
response.text

In [9]:
# 3. Json(str) > list, dict > dataframe
data = response.json()
type(data), data[0] # list형으로 바꿔줌

(list,
 {'localTradedAt': '2022-07-06',
  'closePrice': '2,292.01',
  'compareToPreviousClosePrice': '-49.77',
  'compareToPreviousPrice': {'code': '5', 'text': '하락', 'name': 'FALLING'},
  'fluctuationsRatio': '-2.13',
  'openPrice': '2,330.11',
  'highPrice': '2,332.14',
  'lowPrice': '2,290.33'})

In [12]:
df = pd.DataFrame(data)[["localTradedAt","closePrice"]]
df

,localTradedAt,closePrice
0,2022-07-06,"2,292.01"
1,2022-07-05,"2,341.78"
2,2022-07-04,"2,300.34"
3,2022-07-01,"2,305.42"
4,2022-06-30,"2,332.64"
5,2022-06-29,"2,377.99"
6,2022-06-28,"2,422.09"
7,2022-06-27,"2,401.92"
8,2022-06-24,"2,366.60"
9,2022-06-23,"2,314.32"


In [23]:
# 4. 함수 만들기
# params : pagesize, page
def stock_price(pagesize, page, code="KOSPI"):
    """
    This function is crawling stock price from naver webpage.

    Args:
        pagesize (int): one page size
        page (int): page number
        code (str, optional): kospi or kosdaq. Defaults to "KOSPI".

    Returns:
        DataFrame : display Date, Price
    """
    url = f"https://m.stock.naver.com/api/index/{code}/price?pageSize={pagesize}&page={page}"
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data)
    return df[["localTradedAt", "closePrice"]]

In [21]:
df = stock_price(30, 1)
df.head(2)

,localTradedAt,closePrice
0,2022-08-03,"2,453.34"
1,2022-08-02,"2,439.62"


* **코스닥 정보가져오기**

In [42]:
kospi = stock_price(60, 1)
kosdaq = stock_price(60, 1, "KOSDAQ")

**docstring 사용**

In [25]:
# docstring : 함수의 사용방법 등의 정보 작성
# help(), shift+tab 사용
help(stock_price)

Help on function stock_price in module __main__:

stock_price(pagesize, page, code='KOSPI')
    This function is crawling stock price from naver webpage.
    
    Args:
        pagesize (int): one page size
        page (int): page number
        code (str, optional): kospi or kosdaq. Defaults to "KOSPI".
    
    Returns:
        DataFrame : display Date, Price



### 원달러 환율 데이터 수집
- 네이버 증권 -> 시장지표 미국 USD -> 일별시세

In [26]:
# 최근 60일치 원달러 환율 데이터 수집
# 1. 웹서비스 분석 : URL
# 2. requests > response > JSON
# 3. JSON > DataFrame

In [33]:
url = "https://api.stock.naver.com/marketindex/exchange/FX_USDKRW/prices?page=1&pageSize=60"

In [ ]:
response=requests.get(url)
response.text

In [37]:
data = response.json()
usd = pd.DataFrame(data)[["localTradedAt","closePrice"]]
usd

,localTradedAt,closePrice
0,2022-08-03,"1,309.70"
1,2022-08-02,"1,312.00"
2,2022-08-01,"1,305.00"
3,2022-07-29,"1,304.50"
4,2022-07-28,"1,302.00"


In [40]:
# 데이터분석
# 상관관계분석 : 두 데이터 집합 사이에 어떤 관계가 있는지 확인하는 분석방법
# 원달러 환율리 높으면 코스피, 코스닥 지수가 낮다. > 음의 상관관계를 갖는다.

In [41]:
# 피어슨 상관계수 : df:corr()
# 1과 가까울수록 강한 양의 상관관계
# -1과 가까울수록 강한 음의 상관관계
# 0과 가까울수록 관계가 없다.

In [46]:
df = kospi.copy()
df["kosdaq"] = kosdaq["closePrice"]
df["usd"] = usd["closePrice"]
df = df.rename(columns={"closePrice":"kospi"})
df.head()

,localTradedAt,kospi,kosdaq,usd
0,2022-08-03,"2,453.88",813.05,"1,309.70"
1,2022-08-02,"2,439.62",804.34,"1,312.00"
2,2022-08-01,"2,452.25",807.61,"1,305.00"
3,2022-07-29,"2,451.50",803.62,"1,304.50"
4,2022-07-28,"2,435.27",798.32,"1,302.00"


In [55]:
# 컬럼의 데이터 타입 변경 : str > float
# df[cloumn].apply() : 모든 데이터를 함수에 대입한 결과를 출력
df["kospi"] = df["kospi"].apply(lambda data: float(data.replace(",","")))
df["kosdaq"] = df["kosdaq"].apply(lambda data: float(data.replace(",","")))
df["usd"] = df["usd"].apply(lambda data: float(data.replace(",","")))


In [58]:
df[['kospi','kosdaq','usd']].corr()

,kospi,kosdaq,usd
kospi,1.000000,0.984026,-0.878535
kosdaq,0.984026,1.000000,-0.821457
usd,-0.878535,-0.821457,1.000000
